In [1]:
from tf_trt_models.classification import download_classification_checkpoint

checkpoint_path = download_classification_checkpoint('inception_v2')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
from tf_trt_models.classification import build_classification_graph

frozen_graph, input_names, output_names = build_classification_graph(
    model='inception_v2',
    checkpoint=checkpoint_path,
    num_classes=1001
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./inception_v2/inception_v2.ckpt
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 277 variables.
INFO:tensorflow:Converted 277 variables to const ops.


In [3]:
import tensorflow.contrib.tensorrt as trt

trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP16',
    minimum_segment_size=50
)

INFO:tensorflow:Running against TensorRT version 5.0.6


In [4]:
import tensorflow as tf
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

tf_sess = tf.Session(config=tf_config)

tf.import_graph_def(trt_graph, name='')

tf_input = tf_sess.graph.get_tensor_by_name(input_names[0] + ':0')
tf_output = tf_sess.graph.get_tensor_by_name(output_names[0] + ':0')

In [19]:
MODEL = 'inception_v1'
CHECKPOINT_PATH = 'inception_v1.ckpt'
NUM_CLASSES = 1001
LABELS_PATH = './data/imagenet_labels_%d.txt' % NUM_CLASSES
IMAGE_PATH = 'examples/classification/data/dog-yawning.jpg'

In [21]:
from PIL import Image
import urllib
image = Image.open(IMAGE_PATH)
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.imshow(image)

width = int(tf_input.shape.as_list()[1])
height = int(tf_input.shape.as_list()[2])

import numpy as np
image = np.array(image.resize((width, height)))

In [29]:
import urllib

In [22]:
output = tf_sess.run(tf_output, feed_dict={
    tf_input: image[None, ...]
})

scores = output[0]

In [24]:
import sys
import os
LABELS_PATH = 'examples/classification/data/imagenet_labels_%d.txt' % NUM_CLASSES
with open(LABELS_PATH, 'r') as f:
    labels = f.readlines()

top5_idx = scores.argsort()[::-1][0:5]

for i in top5_idx:
    print('(%3f) %s' % (scores[i], labels[i]))

(0.371518) cocker spaniel, English cocker spaniel, cocker

(0.237076) golden retriever

(0.025085) Blenheim spaniel

(0.020635) clumber, clumber spaniel

(0.013962) Labrador retriever



In [26]:
IMAGE_PATH = 'examples/classification/data/warriors.jpg'
!wget 'https://cdn.vox-cdn.com/thumbor/rC0mlBATZdoDW1tEa44P6431sGc=/0x0:3683x2455/1200x800/filters:focal(1623x234:2211x822)/cdn.vox-cdn.com/uploads/chorus_image/image/63273148/usa_today_12005182.0.jpg' -O {IMAGE_PATH}

--2019-06-26 06:22:23--  https://cdn.vox-cdn.com/thumbor/rC0mlBATZdoDW1tEa44P6431sGc=/0x0:3683x2455/1200x800/filters:focal(1623x234:2211x822)/cdn.vox-cdn.com/uploads/chorus_image/image/63273148/usa_today_12005182.0.jpg
Resolving cdn.vox-cdn.com (cdn.vox-cdn.com)... 151.101.188.124
Connecting to cdn.vox-cdn.com (cdn.vox-cdn.com)|151.101.188.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116962 (114K) [image/jpeg]
Saving to: 'examples/classification/data/warriors.jpg'

examples/classifica 100%[===================>] 114.22K  --.-KB/s    in 0.1s    

2019-06-26 06:22:23 (1.08 MB/s) - 'examples/classification/data/warriors.jpg' saved [116962/116962]



In [34]:
matplotlib.use('Agg')
image = Image.open(IMAGE_PATH)

plt.imshow(image)
width = int(tf_input.shape.as_list()[1])
height = int(tf_input.shape.as_list()[2])


image = np.array(image.resize((width, height)))
#image_resized = np.array(image.resize((300, 300)))
image = np.array(image)

In [35]:
output = tf_sess.run(tf_output, feed_dict={
    tf_input: image[None, ...]
})

scores = output[0]

In [36]:
LABELS_PATH = 'examples/classification/data/imagenet_labels_%d.txt' % NUM_CLASSES
with open(LABELS_PATH, 'r') as f:
    labels = f.readlines()

top5_idx = scores.argsort()[::-1][0:5]

for i in top5_idx:
    print('(%3f) %s' % (scores[i], labels[i]))

(0.802698) basketball

(0.010466) volleyball

(0.007480) parallel bars, bars

(0.006784) balance beam, beam

(0.005963) horizontal bar, high bar

